In [2]:
import json
import os
from PIL import Image

In [3]:
# Функция для нормализации координат
def normalize_coordinates(x, y, width, height, img_width, img_height):
    x_center = (x + width / 2) / img_width
    y_center = (y + height / 2) / img_height
    width /= img_width
    height /= img_height
    return x_center, y_center, width, height

In [11]:
# Загрузка данных VGG
vgg_json_path = './mark29_39_title.json'
with open(vgg_json_path, 'r', encoding='utf-8') as f:
    vgg_data = json.load(f)

In [13]:
# Путь для сохранения аннотаций YOLO
yolo_annotations_path = './yolo_annotations'
os.makedirs(yolo_annotations_path, exist_ok=True)

In [14]:
# Обработка каждого изображения и его аннотаций
for img_key, img_data in vgg_data.items():
    img_filename = img_data['filename']
    img_path = os.path.join('./rtitle', img_filename)
    
    # Получение ширины и высоты изображения с помощью Pillow
    with Image.open(img_path) as img:
        img_width, img_height = img.size

    yolo_annotation_lines = []

    for region in img_data['regions']:
        x = region['shape_attributes']['x']
        y = region['shape_attributes']['y']
        width = region['shape_attributes']['width']
        height = region['shape_attributes']['height']

        x_center, y_center, norm_width, norm_height = normalize_coordinates(x, y, width, height, img_width, img_height)
        yolo_annotation_lines.append(f"0 {x_center} {y_center} {norm_width} {norm_height}")

    # Сохранение аннотации в формате YOLO
    annotation_filename = os.path.splitext(img_filename)[0] + '.txt'
    with open(os.path.join(yolo_annotations_path, annotation_filename), 'w', encoding='utf-8') as f:
        f.write("\n".join(yolo_annotation_lines))